## Geographically Weighted Correlation Coefficient

### Import relevant dependencies

In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import libpysal as ps
from mgwr.gwr import GWR, MGWR, GWRResults
from mgwr.sel_bw import Sel_BW


georgia_data = pd.read_csv(ps.examples.get_path('GData_utm.csv'))
georgia_shp = gp.read_file(ps.examples.get_path('G_utm.shp'))

g_y = georgia_data['PctBach'].values.reshape((-1,1))
g_X = georgia_data[['PctFB']].values
u = georgia_data['X']
v = georgia_data['Y']


g_coords = list(zip(u,v))
# georgia_data.head()

#### Standardization Routine

g_X_std = (g_X - g_X.mean(axis=0)) / g_X.std(axis=0) 

g_y_std = (g_y - g_y.mean(axis=0)) / g_y.std(axis=0)

In [7]:
g_y = georgia_data['PctBach'].values.reshape((-1,1))
g_X = georgia_data[['PctFB']].values
u = georgia_data['X']
v = georgia_data['Y']


g_coords = list(zip(u,v))

gwr_results = GWR(g_coords, g_y, g_X, 60, constant=False).fit()

gwr_results.summary()

Model type                                                         Gaussian
Number of observations:                                                 159
Number of covariates:                                                     1

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                           7595.738
Log-likelihood:                                                    -532.993
AIC:                                                               1067.986
AICc:                                                              1070.063
BIC:                                                               6794.852
R2:                                                                  -0.481
Adj. R2:                                                             -0.481

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

### GW - Correlation Coefficient


$$
   \frac{(x_j - \overline x_i)(y_j - \overline y_i) w_ij}{\sqrt (x_j - \overline x_i)^2 . \sqrt (y_j - \overline y_i)^2 }\
$$

From Page 162-163 of [GWR - The analysis of spatially varying relationships](https://www.academia.edu/33626785/Geographically_Weighted_Regression_The_Analysis_of_Spatially_Varying_Relationships_Wiley_2002)


#### Implementation

In [4]:
# wi =  gwr_results.model._build_wi(0, 60).reshape(-1,1)
# wi = np.sqrt(wi)

# xcc = g_X
# xcc_mean = (xcc).mean(axis=0)

# ycc = g_y
# ycc_mean = (ycc).mean(axis=0)

# numerator = (((xcc - xcc_mean) * (ycc - ycc_mean)))
# denom = (np.sqrt((xcc - xcc_mean)**2)) * (np.sqrt((ycc - ycc_mean)**2))

# r = (numerator/denom)
# # print(r)
# r.mean(axis=0)

### Local OLS and CorrCoef FOR 1 location (data point)

In [12]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr

scaler = StandardScaler()

wi_OLS =  gwr_results.model._build_wi(1, 60).reshape(-1,1)

# wi_OLS = np.sqrt(wi_OLS)

x = g_X
y= g_y

xi = x * wi_OLS
yi = y * wi_OLS

corr_coef, _ = pearsonr(xi[:,0].flatten(), yi.flatten())
print(corr_coef)

xi_std = scaler.fit_transform(xi)
yi_std = scaler.fit_transform(yi)


sm.OLS(yi_std, xi_std).fit().summary()

0.7824826835972309


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.612
Model:                            OLS   Adj. R-squared (uncentered):              0.610
Method:                 Least Squares   F-statistic:                              249.5
Date:                Tue, 27 Sep 2022   Prob (F-statistic):                    2.51e-34
Time:                        02:30:08   Log-Likelihood:                         -150.29
No. Observations:                 159   AIC:                                      302.6
Df Residuals:                     158   BIC:                                      305.6
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.7825      0.050     15.796      0.000       0.685       0.880
==============================================================================
Omnibus:                       69.375   Durbin-Watson:                   2.336
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              213.110
Skew:                           1.764   Prob(JB):                     5.29e-47
Kurtosis:                       7.441   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
gwr_results.params[0]  # So its not the same thing as what I am getting because inside GWR is still using the old methodology
                        # of doing things. I need to go and update the code of GWR to get the output I am having as coefficient
                        # in the above cell. What I am having above is the correct thing actuallu. 

array([0.72790056])

#### Global correlation coefficients

In [6]:
from scipy.stats import pearsonr

# for i in range(3):
#     corr, u = pearsonr(g_X[:,i], g_y.flatten())
    
#     print(corr)

corr, u = pearsonr(g_y.flatten(), g_X.flatten() )
corr

0.6719466884233966

In [ ]:
def _compute_betas_gwr(y, x, wi):
    """
    compute MLE coefficients using iwls routine

    Methods: p189, Iteratively (Re)weighted Least Squares (IWLS),
    Fotheringham, A. S., Brunsdon, C., & Charlton, M. (2002).
    Geographically weighted regression: the analysis of spatially varying relationships.
    """

    xw = (x * wi).T  # weight before standardization -> result is the weighted design matrix

    xw_stdz = scaler.fit_transform(xw)   # standardize the design matrix -- after weighting
    
    x_stdz = scaler.fit_transform(x)     # standardize x 
                    
    y = scaler.fit_transform(y)          # standardize the y


    xtx = np.dot(xw_stdz, x_stdz)
    xtx_inv_xt = linalg.solve(xtx, xw_stdz)
    betas = np.dot(xtx_inv_xt, y)
    return betas, xtx_inv_xt